In [152]:
import sys
import itertools
import heapq
import copy

import numpy as np
import pandas as pd

from numpy import linalg as LA

In [153]:
elems = pd.read_csv('tree_elements.csv')
elems.head()

,Label,Type,Tags,Description,Coaching Year Retired,Coaching Year Started,Current Position,First,Last,Playing Year Retired,Playing Year Started College,Wikipedia info,closeness,metrics::last,year retired
0,Rick Minter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.225974,0.225974,NaN
1,Ernie Zampese,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.231947,0.231947,NaN
2,Frank Cignetti,Coach,NaN,NaN,NaN,1989.0,NaN,NaN,NaN,NaN,NaN,NaN,0.253364,0.253364,NaN
3,Mike Sewak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Mark Stoops,Coach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.206857,0.206857,NaN


In [154]:
edges = pd.read_csv('tree_connections.csv')
edges.head()

,From,To,Label,Type,Tags,Description,Coaching Year Retired,Coaching Year Started,Current Position,First,Last,Playing Year Retired,Playing Year Started College,Wikipedia info,closeness,metrics::last,year retired
0,Al Davis,Tom Flores,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Andy Reid,Leslie Frazier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sid Gillman,Don Coryell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Greg Schiano,Dave Wannstedt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dick Vermeil,Chuck Knox,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
dim = (len(elems.index))
adj = np.zeros((dim, dim), dtype=int)
adj

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [156]:
elems[elems['Label']=="Bill Parcells"].index.values[0]

349

In [157]:
graph_dict = dict([(coach, []) for coach in elems['Label']])

# currently undirected
for index, edge in edges.iterrows():
    coach1 = edge[0]
    coach2 = edge[1]
    coach1_index = elems[elems['Label']==coach1].index.values[0]
    coach2_index = elems[elems['Label']==coach2].index.values[0]
    adj[coach1_index][coach2_index] = 1
    adj[coach2_index][coach1_index] = 1
    graph_dict[coach1].append(coach2)
    graph_dict[coach2].append(coach1)
    

In [158]:
adj

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [159]:
def get_degree_centrality(adj):
    centralities = [sum(row) for row in adj]
    return centralities


In [251]:
def get_top_n_coaches_by_centrality(n, centralityList, reverse=True):
    sorted_cent = sorted(centralityList, reverse=reverse)
    coaches = []
    prev_num = -1
    for i in range(n):
        if sorted_cent[i] != prev_num:
            central_coach_indices = np.where(centralityList==sorted_cent[i])[0]
            for index in central_coach_indices:
                coaches.append(elems.loc[index]['Label'])
            prev_num = sorted_cent[i]
        else: continue
    return coaches

In [252]:
deg_cents = get_degree_centrality(adj)
get_top_n_coaches_by_centrality(15, deg_cents)

['Bo Schembechler',
 'Bill Belichick',
 'Nick Saban',
 'Jim Harbaugh',
 'Woody Hayes',
 'Bill Parcells',
 'Bear Bryant',
 'Pete Carroll',
 'Jon Gruden',
 'John Harbaugh',
 'Bruce Arians',
 'Barry Switzer',
 'Marvin Lewis',
 'Hayden Fry',
 'Marty Schottenheimer',
 'Sid Gillman']

In [253]:
# this is only for a connected graph which explains why this looks horrid
def get_eig_centrality(adj):
    evalues, evectors = LA.eig(adj)
    return [np.absolute(w) for w in evectors[0]]

In [254]:
eig_cents = get_eig_centrality(adj)
get_top_n_coaches_by_centrality(15, eig_cents)

['Mike Gundy',
 'Jim Zorn',
 'Pat Dye',
 'Gary Tranquill',
 'Jim McElwain',
 'Mike Tice',
 'Frank A. Mason',
 'Norv Turner',
 'Frank Reich',
 'Sam Sanders',
 'Glen Mason',
 'Freddie Kitchens',
 'John Fox',
 'Frank W. Thomas',
 'Dan Reeves']

In [255]:
def find_shortest_path(graph, start, end, path=[]):
    """
    __source__='https://www.python.org/doc/essays/graphs/'
    __author__='Guido van Rossum'
    """
    path = path + [start]
    if start == end:
        return path
    if not start in graph:
        return None
    shortest = None
    for node in graph[start]:
        if node not in path:
            newpath = find_shortest_path(graph, node, end, path)
            if newpath:
                if not shortest or len(newpath) < len(shortest):
                    shortest = newpath
    return shortest

# EXAMPLE TEST:
# if __name__=='__main__':
#     graph = {'A': ['B', 'C'],
#              'B': ['C', 'D'],
#              'C': ['D'],
#              'D': ['C'],
#              'E': ['F'],
#              'F': ['C']}
#     print(find_shortest_path(graph,'A','D'))
#     # ['A', 'B', 'D']
#     print(len(find_shortest_path(graph,'A','D')))
#     # 3

In [256]:
class SimpleVertex(object):
    
    def __init__(self, name, neighbors):
        self.name = name
        self.neighbors = neighbors
        self.dist = sys.maxsize
        self.prev = []
    
    def __str__(self):
        s = self.name
        s += "\n  Neighbors: {}".format(neighbors)
        return s

def my_dijkstra_no_priority(elem_dict, source_vertex_name):
    '''Dijkstras Algo for finding the distance between two points on a graph
    Based on the pseudocode here: https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm#Pseudocode'''
    
    static_vertex_set = []
    name_vertex_dict = {}
    
    for key in elem_dict:
        vertex = SimpleVertex(key, elem_dict[key])
        static_vertex_set.append(vertex)
        name_vertex_dict[key] = vertex
        
    name_vertex_dict[source_vertex_name].dist = 0
    
    Q_vertex_set = sorted(copy.copy(static_vertex_set), key=lambda v: v.dist)
    while len(Q_vertex_set) > 0:
        Q_vertex_set = sorted(Q_vertex_set, key=lambda v: v.dist)
        u = Q_vertex_set.pop(0)
        
        for node in u.neighbors:
            alt = u.dist + 1 # replace 1 with length(u,v) if distances are weighted
            node_obj = name_vertex_dict[node]
            if alt < node_obj.dist:
                node_obj.dist = alt
                node_obj.prev = u
                
    # return list of distances and previous nodes
    all_dists = [v.dist for v in static_vertex_set]
    all_prevs = [v.prev for v in static_vertex_set]
    
    return all_dists, all_prevs


In [260]:
# UNDIRECTED
def distance_matrix(elem_dict, elems, edges):
    dist_mat = np.zeros((len(elems.index), len(elems.index)))
    
#     combos = [combo for combo in itertools.combinations_with_replacement(list(elems['Label']), 2) if combo[0] != combo[1]]
#     numDone = 0
#     for pair in combos:
#         coach1_index = elems[elems['Label']==pair[0]].index.values[0]
#         coach2_index = elems[elems['Label']==pair[1]].index.values[0]
        
        
#         pathlength = find_shortest_path(elem_dict, pair[0], pair[1])
        
#         dist[coach1_index][coach2_index] = pathlength
#         dist[coach2_index][coach1_index] = pathlength
#         numDone += 1
#         if numDone % 5 == 0:
#             print("Found {} pathlengths".format(numDone))
    i = 0
    for coach in elems['Label']:
        dists, prevs = my_dijkstra_no_priority(elem_dict, coach)
        dist_mat[i] = dists
        i += 1
    
    return dist_mat
        

def get_closeness_centrality(elems, elem_dict, edges):
    coaches = elems['Label']
    d = distance_matrix(elem_dict, elems, edges)
    closenesses = [sum(row) for row in d]
    return closenesses
    

In [261]:
closeness = get_closeness_centrality(elems, graph_dict, edges)

In [262]:
n = 15
closeness_names = get_top_n_coaches_by_centrality(n, closeness, reverse=False)
print("Name                Closeness")
for i in range(n):
    print("{0}{1}{2}".format(closeness_names[i], "".join([" " for x in range(20-len(closeness_names[i]))]), closeness[i]))

Name                Closeness
Nick Saban          2197.0
Bo Schembechler     2433.0
Jim Harbaugh        2189.0
Woody Hayes         3317.0
Earle Bruce         2536.0
Pete Carroll        2046.0
Bill Belichick      2638.0
Sid Gillman         2413.0
Frank Maloney       2398.0
Mike Mularkey       3279.0
Dan Quinn           2309.0
Monte Kiffin        2206.0
Cam Cameron         2331.0
Freddie Kitchens    2165.0
Bill Parcells       2317.0
